# Overview

Make sure that this notebook is run in the Conda environment setup for this project, as described in [Readme.md](README.md)

This notebook runs the code for performing schooling experiments. 

# Parameters

In [ ]:
# Prefix for video files
moviefile_prefix = "SCHOOL_S001_S001_T"

# Address for the Enttec USB device (see above)
hw_address = "/dev/cu.usbserial-EN373474"

# Define IP address of smart switch (see README for how to find at command line)
LED_IP = "192.168.0.104"

# Packages

In [ ]:
from DMXEnttecPro import Controller
import numpy as np
import def_runexperiments as re
import multiprocess, os
# import pandas as pd

# This is necessary for using multiprocess on a Mac
if os.name == "posix":
        multiprocess.set_start_method('spawn')

## Project paths

You can find instructions for defining hw_address in [README.md](README.md). You will need to be connected to the fileserver ("vortex") to access the log_path.

In [ ]:
# Path to time-code audio file
aud_path = "/Users/experimentalist/Documents/Projects/wake_tracking/timecode.wav"

# Path to log csv file
log_path = "/Volumes/schooling/TRex/recording_log.csv"

# Check address of log file
if not os.path.isfile(log_path):
    raise OSError("log_path not found at " + log_path)

# Check address of audio file
if not os.path.isfile(aud_path):
    raise OSError("aud_path not found at " + aud_path)

# log = pd.read_csv(log_path)
# print(log)

## Initialize hardware

Run these lines for all of the programs that you want to run below.

In [ ]:
#  Initialize hardware
dmx = Controller(hw_address, auto_submit=True, dmx_size=32)

# Set output rate at maximum
dmx.set_dmx_parameters(output_rate=0)

# Basic hardware control

## Change fixed light intensity
Note that if the Luxli LED lamp becomes unresponsive, then disconnect the DMX connection at the Enttec USB box, then power off the power supply to the light, turn the power supply back on, then power up the light (it should show the interactive screen), and then reconnect the DMX.

In [ ]:
# Specify light intensity (from 0 to 1)
light_intensity = 0

# Sets DMX channel 1 to max 255 (Channel 1 is the intensity)
dmx.set_channel(1, int(light_intensity*255))  

## Turn on LED array

In [ ]:
os.system('kasa --host ' + LED_IP + ' on')

## Turn off LED array

In [ ]:
os.system('kasa --host ' + LED_IP + ' off')

# Single-Experiment Programs

Select which of the cells below that make sense for the experiment that you want to run.

## Lights on for fixed duration

In [ ]:
# What light level to return to after the program
end_light_level = 0

# list of light levels
light_level = np.array([1])

# Duration that each level is fixed (s)
light_dur = np.array([2])

# Run control program
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, trig_video=True, echo=False, plot_data=True, 
    movie_prefix=moviefile_prefix, LED_IP=LED_IP)

# Set ending log intensity
dmx.set_channel(1, int(end_light_level*255))  

# print(df)

## Lights on -> Lights off

Ramp down light intensity at a constant rate.

In [ ]:
# list of light levels
light_level = np.array([1, 0])

# Duration that each level is fixed (s)
light_dur = np.array([10, 10])

# Duration for the changes in light level
ramp_dur = np.array([0.5])

# Play control levels into Enttec DMX
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, ramp_dur=ramp_dur, trig_video=True, echo=False, plot_data=True,
    movie_prefix=moviefile_prefix, LED_IP=LED_IP)

# print(df)

## Lights off -> Lights on

Ramp down light intensity at a constant rate.

In [ ]:
# list of light levels
light_level = np.array([0, 1])

# Duration that each level is fixed
light_dur = np.array([5, 5])

# Duration for the changes in light level
ramp_dur = np.array([2])

# Play control levels into Enttec DMX
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, ramp_dur=ramp_dur, trig_video=True, echo=False, plot_data=True,
    movie_prefix=moviefile_prefix, LED_IP=LED_IP)

# print(df)

## Ramp lights down and then up

Changing light intensity at a constant rate.

In [ ]:
# list of light levels
light_level = np.array([1, 0, 0.5])

# Duration that each level is fixed
light_dur = np.array([1, 2, 1])

# Duration for the changes in light level
ramp_dur = np.array([0.5, 0.5])

# Play control levels into Enttec DMX
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, ramp_dur=ramp_dur, trig_video=True, echo=False, plot_data=True,
     movie_prefix=moviefile_prefix, LED_IP=LED_IP)

# print(df)

# Sequence of experiments

## Vary ramp duration

In [ ]:
# list of starting and ending light levels
light_level = np.array([1, 0])

# Duration that each level is fixed at the start and end of experiment (s)
light_dur = np.array([10, 10])

# Duration for the ramp of changes in light level (s)
min_rampdur = 0.1
max_rampdur = 1.5

# Number of experiments to run
num_expts = 3

# Period of time bwteen expts (min)
btwn_period = 0.25

# Play control levels into Enttec DMX
re.seq_vary_rampdur(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, num_ramp=num_expts, min_rampdur=min_rampdur, max_rampdur=max_rampdur, btwn_period=btwn_period, movie_prefix=moviefile_prefix, LED_IP=LED_IP)